# Interactive Spark Session on Synapse Spark Pool

### Install the azureml-synapse package on your compute instance.

We will pip install the package *azureml-synapse*, but note if you chose to use jupyter lab, then you would need to additionally need to run the command: *!jupyter lab build --minimize=False*

In [1]:
!pip install -U "azureml-synapse"

     |████████████████████████████████| 2.8 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 82.3 MB/s eta 0:00:01
     |████████████████████████████████| 442 kB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 154 kB 78.9 MB/s eta 0:00:01
     |████████████████████████████████| 109 kB 83.4 MB/s eta 0:00:01
  Created wheel for sparkmagic: filename=sparkmagic-0.16.0-py3-none-any.whl size=57247 sha256=c70d1085636c921576159282af83309c2daee1e7a5290ac5b04929fefff53bd3
  Stored in directory: /home/azureuser/.cache/pip/wheels/69/94/d4/94d1e290dc0c3965449c6d1832ef9b8a86c81bca7bf4e632c7
  Created wheel for hdijupyterutils: filename=hdijupyterutils-0.16.0-py3-none-any.whl size=7697 sha256=00d18aeb9509f3ee15a9f160042a9de6905fab922402adfb745572f8a5281196
  Stored in directory: /home/azureuser/.cache/pip/wheels/21/77/5f/ed23a25516a7d1f93f04fd9953e1985a61694ed9bc3715ccae
  Created wheel for autovizwidget: filename=autovizwidget-0.19.1-py3-none-any.whl size=14544 sh

### Restart your kernal and refresh the web page after doing the install. 

This is due to the fact that package will install the line magic function `%synsapse` that will leverage.
If you try to run the next command, and fail, you probably skipped this step. 

In [1]:
# show help
%synapse ?

::

  %synapse [-s SUBSCRIPTION_ID] [-r RESOURCE_GROUP] [-w WORKSPACE_NAME]
               [-f CONFIG_FILE] [-c COMPUTE_TARGET]
               [--driver-memory DRIVER_MEMORY] [--driver-cores DRIVER_CORES]
               [--executor-memory EXECUTOR_MEMORY]
               [--executor-cores EXECUTOR_CORES] [-n NUM_EXECUTORS]
               [-t SESSION_TIMEOUT] [--start-timeout START_TIMEOUT]
               [-e ENVIRONMENT] [--environment-version ENVIRONMENT_VERSION]
               [command [command ...]]

Magic to execute spark remotely against a Synapse Spark pool.

Sub commands:
    start: Start a Livy session against target AML Synapse compute.
        You can set spark config in the magic body by json format. e.g.

        .. code-block:: python

            %%synapse start --compute-target synapse_compute
            {
                "spark.yarn.appMasterEnv.xxx": "xxx",
                "spark.executorEnv.xxx": "xxx",
                "spark.yarn.maxAppAttempts": 1
            }

   

## Start your spark Session

use Synapse compute linked to the Compute Instance's workspace with an aml envrionment.
conda dependencies specified in the environment will be installed before the spark session started.

Starting the session will take several minutes as it is installing your environment packages into your Spark Session

In [4]:
synapse_compute_name=os.getenv("SYNAPSE_COMPUTE_NAME", "aml-mmsparkpool")

In [5]:
synapse_compute_name

'aml-mmsparkpool'

In [6]:
%synapse start -c $synapse_compute_name -e AzureML-Minimal

Synapse Spark pool now only supports fixed Python version, you can print "sys.version_info" in your script to check current Python version.
Starting session 'aml_notebook_392851' under environment 'AzureML-Minimal', this may take several minutes .......................................................................................................... Succeeded!


In [ ]:
# Connecting to Data

Currently 3 types of datastores are supported with Synapse Spark
- Blob Storage - providing credentials
- Adls (Gen 1 & 2) - both credential & credential less.

## Example 1: Data loading by HDFS path

### Read data from blob

```python
# setup access key or sas token

sc._jsc.hadoopConfiguration().set("fs.azure.account.key.<storage account name>.blob.core.windows.net", "<acess key>")
sc._jsc.hadoopConfiguration().set("fs.azure.sas.<container name>.<storage account name>.blob.core.windows.net", "sas token")

df = spark.read.parquet("wasbs://<container name>@<storage account name>.blob.core.windows.net/<path>")
```

### Read data from ADLSGen1

```python
# setup service pricinpal which has access of the data
# If no data Credential is setup, the user identity will be used to do access control

sc._jsc.hadoopConfiguration().set("fs.adl.account.<storage account name>.oauth2.access.token.provider.type","ClientCredential")
sc._jsc.hadoopConfiguration().set("fs.adl.account.<storage account name>.oauth2.client.id", "<client id>")
sc._jsc.hadoopConfiguration().set("fs.adl.account.<storage account name>.oauth2.credential", "<client secret>")
sc._jsc.hadoopConfiguration().set("fs.adl.account.<storage account name>.oauth2.refresh.url", "https://login.microsoftonline.com/<tenant id>/oauth2/token")

df = spark.read.csv("adl://<storage account name>.azuredatalakestore.net/<path>")
```

### Read data from ADLSGen2

```python
Read data from Adlsgen2

# setup service pricinpal which has access of the data
# If no data Credential is setup, the user identity will be used to do access control

sc._jsc.hadoopConfiguration().set("fs.azure.account.auth.type.<storage account name>.dfs.core.windows.net","OAuth")
sc._jsc.hadoopConfiguration().set("fs.azure.account.oauth.provider.type.<storage account name>.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
sc._jsc.hadoopConfiguration().set("fs.azure.account.oauth2.client.id.<storage account name>.dfs.core.windows.net", "<client id>")
sc._jsc.hadoopConfiguration().set("fs.azure.account.oauth2.client.secret.<storage account name>.dfs.core.windows.net", "<client secret>")
sc._jsc.hadoopConfiguration().set("fs.azure.account.oauth2.client.endpoint.<storage account name>.dfs.core.windows.net", "https://login.microsoftonline.com/<tenant id>/oauth2/token")

df = spark.read.csv("abfss://<container name>@<storage account>.dfs.core.windows.net/<path>")
```

In [76]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))
print(ws)

default_ds = ws.get_default_datastore()

default_ds.upload_files(files=['./data/train.csv', './data/test.csv'], # Upload the diabetes csv files in /data
                       target_path='titanic/', # Put it in a folder path in the datastore
                       overwrite=True, # Replace existing files of the same name
                       show_progress=True)

Ready to use Azure ML 1.34.0 to work with mm-aml-dev
Workspace.create(name='mm-aml-dev', subscription_id='5da07161-3770-4a4b-aa43-418cbbb627cf', resource_group='mm-machine-learning-dev-rg')
Uploading an estimated of 2 files
Uploading ./data/train.csv
Uploaded ./data/train.csv, 1 files out of an estimated total of 2
Uploading ./data/test.csv
Uploaded ./data/test.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_9d31278579ba484f9424ba37f08eee34

In [79]:
%%synapse

import azureml.core
from azureml.core import Dataset
import os, shutil

print(azureml.core.VERSION)

from azureml.core import Workspace, Dataset
ws = Workspace.get(name='mm-aml-dev', subscription_id='5da07161-3770-4a4b-aa43-418cbbb627cf', resource_group='mm-machine-learning-dev-rg')
#ds = Dataset.get_by_name(ws, "<tabular dataset name>")

# Get the default datastore
default_ds = ws.get_default_datastore()

tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'titanic/train.csv'))
df = tab_data_set.to_spark_dataframe()

print('row count:', df.count(),'column count:', len(df.columns))


1.34.0
row count: 891 column count: 12


In [145]:
%%synapse

#from pyspark.sql.functions import col, desc
import pyspark.sql.functions as F
from IPython.display import display

df = spark.read.option("header", "true").csv("wasbs://demo@dprepdata.blob.core.windows.net/Titanic.csv")
df.filter(F.col('Survived') == 1).groupBy('Age').count().orderBy(F.desc('count')).show(5)


+----+-----+
| Age|count|
+----+-----+
|null|   52|
|  24|   15|
|  27|   11|
|  35|   11|
|  22|   11|
+----+-----+
only showing top 5 rows


# Challenge - Cleanse your data

Determine which columns have NULL values and determine as a team what you would like to do with that data.  Save your resulting data to your teams' ADLS Gen2 Storage Location

Helpful Articles for your team's discussion:
[Towards Datascience](https://towardsdatascience.com/machine-learning-with-the-titanic-dataset-7f6909e58280)

Potentially helpful article on [sparkbyexamples](https://sparkbyexamples.com/pyspark/pyspark-find-count-of-null-none-nan-values/#:~:text=In%20PySpark%20DataFrame%20you%20can%20calculate%20the%20count,all%20or%20multiple%20selected%20columns%20of%20PySpark%20DataFrame)



In [146]:
%%synapse

print('row count:', df.count(),'column count:', len(df.columns))

row count: 891 column count: 12


In [147]:
%%synapse
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+


In [148]:
%%synapse
from pyspark.sql.functions import col,isnan,when,count
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+


In [161]:
%%synapse
cols = df.columns
for c in cols:
    value = df.agg({c: 'avg'})
    median = value.collect()[0][0]
    print(c, median)
    if median != None:
        #df = df.na.fill(value=median,subset=[c])
        #df.isNull.fill(value=median,subset=[c])


An error was encountered:
unexpected EOF while parsing (<stdin>, line 8)
  File "<stdin>", line 8
    #df.isNull.fill(value=median,subset=[c])
                                           ^
SyntaxError: unexpected EOF while parsing



In [158]:
%%synapse
from pyspark.sql.functions import col,isnan,when,count
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|    0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+


In [151]:
%%synapse
cabins = df.select('Cabin').rdd.flatMap(lambda x: x).distinct().collect()
cabins

[None, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27', 'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31', 'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47', 'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4', 'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77', 'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D', 'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7', 'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37', 'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104', 'C111', 'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30', 'D20', 'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22', 'C86', 'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19', 'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71', 'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24', 'C90', 'C45', 'E8', 'B101', 'D45', 'C46', 'D30', 'E121', 'D11', 'E77', 'F38', 'B3', 'D6', 'B82 

In [152]:
%%synapse

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def get_deck(s):
     return 'M' if s == None else s[0]
    
df = df.withColumn("Cabin", get_deck("Cabin"))

df.head(5)

[Row(PassengerId='1', Survived='0', Pclass='3', Name='Braund, Mr. Owen Harris', Sex='male', Age='22', SibSp='1', Parch='0', Ticket='A/5 21171', Fare='7.25', Cabin='M', Embarked='S'), Row(PassengerId='2', Survived='1', Pclass='1', Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age='38', SibSp='1', Parch='0', Ticket='PC 17599', Fare='71.2833', Cabin='C', Embarked='C'), Row(PassengerId='3', Survived='1', Pclass='3', Name='Heikkinen, Miss. Laina', Sex='female', Age='26', SibSp='0', Parch='0', Ticket='STON/O2. 3101282', Fare='7.925', Cabin='M', Embarked='S'), Row(PassengerId='4', Survived='1', Pclass='1', Name='Futrelle, Mrs. Jacques Heath (Lily May Peel)', Sex='female', Age='35', SibSp='1', Parch='0', Ticket='113803', Fare='53.1', Cabin='C', Embarked='S'), Row(PassengerId='5', Survived='0', Pclass='3', Name='Allen, Mr. William Henry', Sex='male', Age='35', SibSp='0', Parch='0', Ticket='373450', Fare='8.05', Cabin='M', Embarked='S')]


In [153]:
%%synapse
from pyspark.sql.functions import col,isnan,when,count
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|    0|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+


In [154]:
%%synapse
df.filter(df.Embarked.isNull()).show()

+-----------+--------+------+--------------------+------+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+---+-----+-----+------+----+-----+--------+
|         62|       1|     1| Icard, Miss. Amelie|female| 38|    0|    0|113572|  80|    B|    null|
|        830|       1|     1|Stone, Mrs. Georg...|female| 62|    0|    0|113572|  80|    B|    null|
+-----------+--------+------+--------------------+------+---+-----+-----+------+----+-----+--------+


In [155]:
%%synapse
df = df.dropna(subset=["Embarked"])

df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|    0|       0|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+


In [156]:
%%synapse

print('row count:', df.count(),'column count:', len(df.columns))

row count: 889 column count: 12
